In [1]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import plotly.graph_objects as go

In [2]:
PARTICIPANT = 'TEST_VIDEO'
df = pd.read_csv(f"../Upsampled/{PARTICIPANT}_processed_data.csv")

In [3]:
# Using scipy's find_peaks method to find peaks in the 'right_wrist_speed' column and save 
peaks, _ = find_peaks(df['right_wrist_speed'], height=(np.percentile(df['right_wrist_speed'], 70)))  # height parameter can be adjusted based on your specific needs
# Keep the time values of the peaks
peak_times = df['time_ms'][peaks]

In [4]:
# Initialize empty lists to store onset and offset points
onsets = []
offsets = []
# Define a threshold for onset and offset detection; this can be adjusted based on your needs
onset_threshold = np.percentile(df['right_wrist_speed'], 5)
offset_threshold = np.percentile(df['right_wrist_speed'], 5)

# Loop through each peak to find the corresponding onset and offset
for peak in peaks:
    # Find onset by iterating backwards from the peak until the speed drops below the threshold
    onset = peak
    while onset > 0 and df['right_wrist_speed'][onset] > onset_threshold:
        onset -= 1
    onsets.append(df['time_ms'][onset])
    
    # Find offset by iterating forwards from the peak until the speed drops below the threshold
    offset = peak
    while offset < len(df) - 1 and df['right_wrist_speed'][offset] > offset_threshold:
        offset += 1
    offsets.append(df['time_ms'][offset])

In [5]:
# Pair each onset with the next offset to form complete gestures
events = list(zip(onsets, offsets))
events_df = pd.DataFrame(events, columns=['onset', 'offset'])
events_df.drop_duplicates(subset=['onset', 'offset'], inplace=True)
events_df['duration'] = events_df['offset'] - events_df['onset']
events_df.reset_index(drop=True, inplace=True)
events_df['gesture_id'] = events_df.index + 1

# Merge gestures that are too close together
for i in range(len(events_df) - 1):
    if events_df['onset'][i + 1] - events_df['offset'][i] < 50:
        events_df['offset'][i] = events_df['offset'][i + 1]
        events_df['duration'][i] = events_df['offset'][i] - events_df['onset'][i]
        events_df['gesture_id'][i + 1] = events_df['gesture_id'][i]

df['gesture_id'] = np.nan
for i in range(len(events_df)):
    start = events_df['onset'][i]
    end = events_df['offset'][i]
    df['gesture_id'] = np.where((df['time_ms'] >= start) & (df['time_ms'] <= end), events_df['gesture_id'][i], df['gesture_id'])

In [6]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['time_ms'], y=df['gesture_id'], name='Gesture ID'))
fig.add_trace(go.Scatter(x=df['time_ms'], y=df['right_wrist_speed'], name='Right Wrist Speed', yaxis='y2'))

fig.update_layout(
       yaxis=dict(
              title='Gesture ID',
              titlefont=dict(color='blue'),
              tickfont=dict(color='blue')
       ),
       yaxis2=dict(
              title='Right Wrist Speed',
              titlefont=dict(color='red'),
              tickfont=dict(color='red'),
              overlaying='y',
              side='right'
       ),
       width=1000,
       height=800
)

fig.show()

In [7]:
# Save the new annotated dataframe as a csv file
df.to_csv(f'../Events/{PARTICIPANT}_events_found.csv', index=False)